In [ ]:
# Import Libraries

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains
import random

sleep_time = random.uniform(2, 5)

#Enter Twitter

url ="https://www.twitter.com"

driver = webdriver.Chrome()

driver.get(url)

time.sleep(sleep_time)

#Sign in

sign_in = driver.find_element(By.CSS_SELECTOR, "[data-testid='loginButton']")

sign_in.click()

time.sleep(sleep_time)

#Input Email
# Localiza el campo de texto por su atributo autocomplete

user_field = driver.find_element(By.CSS_SELECTOR, "input[autocomplete='username']")

# Introduce el nombre de usuario
username = "enter your email here"
user_field.send_keys(username)

time.sleep(3)

element = driver.find_element(By.XPATH, "/html/body/div/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/div[6]")

time.sleep(sleep_time)

element.click()

#Input Username

time.sleep(sleep_time)
# Localiza el elemento por su selector CSS
user_bar = driver.find_element(By.XPATH, "/html/body/div/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div[2]/label/div/div[2]/div/input")

username = "enter your username here"
# Introduce el texto en el elemento
user_bar.send_keys(username)
next_bt = driver.find_element(By.XPATH, "/html/body/div/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div/div/div/div/span/span")
next_bt.click()

#Input Password

time.sleep(sleep_time)

# Localiza el elemento por su selector CSS
campo_contraseña = driver.find_element(By.CSS_SELECTOR, "input[name='password']")

password = "enter your password here"
# Introduce el texto en el campo de contraseña
campo_contraseña.send_keys(password)

next_bt = driver.find_element(By.XPATH, "/html/body/div/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div/div/span/span")

next_bt.click()
time.sleep(sleep_time)

search_filed = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div[2]/main/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div/div/form/div[1]/div/div/div/label/div[2]/div/input")

search_user = "enter the target user here" #it has to be with the @
search_filed.send_keys(search_user)

time.sleep(sleep_time)

search_bt = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div[2]/main/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div/div/form/div[2]/div/div[5]/div/div")

search_bt.click()
time.sleep(sleep_time)

### SCRAP

def procesar_elementos_time(time_elements, elementos_procesados):
    for elemento in time_elements:
        fecha_y_hora = elemento.get_attribute("datetime").split("T")
        if fecha_y_hora[0] in elementos_procesados:
            continue  # Si el elemento ya ha sido procesado, saltar al siguiente
        fecha = fecha_y_hora[0]
        dia = fecha.split("-")[2]
        mes = fecha.split("-")[1]
        anno = fecha.split("-")[0]
        texto_formateado = f"{dia} {mes}. {anno}"
        elemento_formateado = f'<time datetime="{fecha_y_hora[0]}T{fecha_y_hora[1]}">{texto_formateado}</time>'
        form_elements.append(elemento_formateado)
        years.append(anno)
        months.append(mes)
        days.append(dia)
        hours_raw.append(fecha_y_hora[1])
        elementos_procesados.add(fecha_y_hora[0])  # Añade la fecha al conjunto de elementos procesados

    return form_elements, years, months, days, hours_raw, elementos_procesados

# Inicializar las listas fuera del bucle
form_elements = []
years = []
months = []
days = []
hours_raw = []
elementos_procesados = set()  # Un conjunto para mantener un registro de los elementos ya procesados

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    time.sleep(sleep_time)  # Espera un tiempo aleatorio antes de enviar la tecla 'Fin'
    driver.find_element(By.XPATH, '//body').send_keys(Keys.END)  # Pulsar la tecla 'Fin'
    time.sleep(sleep_time)  # Espera un momento

    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # Si la altura no cambió, hemos llegado al final de la página
    last_height = new_height

    # Encuentra elementos de tiempo
    time_elements = driver.find_elements(By.XPATH, '//time[contains(@datetime, "T") and contains(text(), " ") and not(ancestor::*[@class="css-175oi2r r-eqz5dr r-1fz3rvf r-1s2bzr4"])]')
    form_elements, years, months, days, hours_raw, elementos_procesados = procesar_elementos_time(time_elements, elementos_procesados)

hours = [hour.split('.')[0] for hour in hours_raw]

### Print complete output
for elemento_formateado in form_elements:
    print(elemento_formateado)

### Print Lists with length to check 
print(years)
print(len(years))

print(months)
print(len(months))

print(days)
print(len(days))

print(hours)
print(len(hours))

### Split Time

hour = [time.split(':')[0] for time in hours]
minute = [time.split(':')[1] for time in hours]
second = [time.split(':')[2] for time in hours]

print("Hour:", hour)
print("Minute:", minute)
print("Second:", second)

## Create and print DataFrame
df = pd.DataFrame({
                    'Day' : days,
                    'Month': months,
                    'Hour' : hour,
                    'Minute' : minute,
                    'Second' : second,
                    'Year' : years
})
print(df)

#€€ Export to .csv
df.to_csv(f"../data/df_{search_user}.csv", index = False)